In [52]:
import numpy
import pandas as pd
import json
#https://pypi.org/project/yahooquery
from yahooquery import Ticker
import datetime
import pyodbc
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import sys, getopt

In [53]:
interval = ''
argv = ['-i', 'daily']

try:
    opts, args = getopt.getopt(argv, "hi:", ["interval="])
except getopt.GetoptError:
    print('tradingdatadownload.py -i <interval>')
    sys.exit(2)

for opt, arg in opts:
    if opt == '-h':
        print('tradingdatadownload.py -i <interval>')
        sys.exit()
    elif opt in ("-i", "--interval"):
        interval = arg.lstrip()

if interval == '':
    print('tradingdatadownload.py -i <interval>')
    sys.exit(2)

try:
    cnxn = pyodbc.connect('Driver={ODBC Driver 18 for SQL Server};Server=tcp:trading-website.database.windows.net,1433;Database=trading-database;Uid=derek;Pwd=Work2@22;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')

    cnxn_string = 'Driver={ODBC Driver 18 for SQL Server};Server=tcp:trading-website.database.windows.net,1433;Database=trading-database;Uid=derek;Pwd=Work2@22;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
except pyodbc.Error as ex:
    sqlstate = ex.args[1]
    print(sqlstate)
    sys.exit(2)


cnxn_url = URL.create("mssql+pyodbc", query={"odbc_connect": cnxn_string})

engine = create_engine(cnxn_url)


In [54]:
sym = pd.read_sql_table("Symbol", engine)
ticker_symbols = sym['Symbol'].values.tolist()
print(ticker_symbols)

['AAPL      ', 'TSLA      ', 'NFLX      ', 'PLTR      ']


In [28]:
if interval == 'weekly':
    tickers = Ticker(ticker_symbols)
    weekly = tickers.history(period='5y', interval='1wk')
    print('Download weekly completed')
    saveToTable(weekly, 'Weekly', cnxn, engine)
    print('Saved to table weekly')

if interval == 'daily':
    downloadDaily(ticker_symbols, cnxn, engine)

if interval == 'hourly':
    tickers = Ticker(ticker_symbols)
    hourly = tickers.history(period='1mo', interval='1h')
    print('Download hourly completed')
    saveToTable(hourly, 'Hourly', cnxn, engine)
    print('Saved to table hourly')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/yahooquery/utils/__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/yahooquery/ticker.py:1333: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["dividends"].fillna(0, inplace=True)


ValueError: cannot insert symbol, already exists

In [55]:
cursor = cnxn.cursor()
cursor.execute('EXEC spProcessDailyData')
cursor.commit()
cursor.close()



In [57]:
for symbol in ticker_symbols:
    ticker = Ticker(symbol)
    daily = ticker.history(period='1y', interval='1d')
    print(f'Downloading {symbol}.')

    daily.to_sql('tmpDaily', engine, if_exists='append',schema='dbo', index=True)

cursor = cnxn.cursor()
cursor.execute('EXEC spProcessDailyData')
cursor.commit()
cursor.close()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/yahooquery/utils/__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/yahooquery/ticker.py:1333: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["dividends"].fillna(0, inplace=True)


ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Column 'symbol' in table 'dbo.Option' is of a type that is invalid for use as a key column in an index. (1919) (SQLExecDirectW)")
[SQL: CREATE INDEX [ix_dbo_Option_symbol] ON dbo.[Option] (symbol)]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [38]:
ticker = Ticker('AAPL')
daily = ticker.history(period='1y', interval='1d')

daily.head(260)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/yahooquery/utils/__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/yahooquery/ticker.py:1333: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["dividends"].fillna(0, inplace=True)


open        high         low       close    volume  \
symbol date                                                                   
AAPL   2024-02-22  183.479996  184.960007  182.460007  184.369995  52292200   
       2024-02-23  185.009995  185.039993  182.229996  182.520004  45119700   
       2024-02-26  182.240005  182.759995  180.649994  181.160004  40867400   
       2024-02-27  181.100006  183.919998  179.559998  182.630005  54318900   
       2024-02-28  182.509995  183.119995  180.130005  181.419998  48953900   
...                       ...         ...         ...         ...       ...   
       2025-02-14  241.250000  245.550003  240.990005  244.600006  40896200   
       2025-02-18  244.149994  245.179993  241.839996  244.470001  48822500   
       2025-02-19  244.660004  246.009995  243.160004  244.869995  32204200   
       2025-02-20  244.940002  246.779999  244.289993  245.830002  32316900   
       2025-02-21  245.949997  248.690002  245.220001  245.550003  53119400   

                     adjclose  dividends  
symbol date                               
AAPL   2024-02-22  183.503540        0.0  
       2024-02-23  181.662231        0.0  
       2024-02-26  180.308624        0.0  
       2024-02-27  181.771713        0.0  
       2024-02-28  180.567398        0.0  
...                       ...        ...  
       2025-02-14  244.600006        0.0  
       2025-02-18  244.470001        0.0  
       2025-02-19  244.869995        0.0  
       2025-02-20  245.830002        0.0  
       2025-02-21  245.550003        0.0  

[251 rows x 7 columns]

In [ ]:
daily["symbol"] = 'AAPL'

In [50]:
daily.index

MultiIndex([('AAPL', 2024-02-22),
            ('AAPL', 2024-02-23),
            ('AAPL', 2024-02-26),
            ('AAPL', 2024-02-27),
            ('AAPL', 2024-02-28),
            ('AAPL', 2024-02-29),
            ('AAPL', 2024-03-01),
            ('AAPL', 2024-03-04),
            ('AAPL', 2024-03-05),
            ('AAPL', 2024-03-06),
            ...
            ('AAPL', 2025-02-07),
            ('AAPL', 2025-02-10),
            ('AAPL', 2025-02-11),
            ('AAPL', 2025-02-12),
            ('AAPL', 2025-02-13),
            ('AAPL', 2025-02-14),
            ('AAPL', 2025-02-18),
            ('AAPL', 2025-02-19),
            ('AAPL', 2025-02-20),
            ('AAPL', 2025-02-21)],
           names=['symbol', 'date'], length=251)